In [10]:
import os
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import DepthwiseConv2D

absProjectDir = Path(os.getcwd()).resolve().parents[0]
projectDir = os.path.relpath(absProjectDir,os.curdir)
load_dotenv(find_dotenv())

trainDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TRAIN_DF"))
testDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TEST_DF"))
testOrigDir = os.path.join(projectDir, os.environ.get("PROC_TEST_ORIG_DIR"))
testAugmDir = os.path.join(projectDir, os.environ.get("PROC_TEST_AUG_DIR"))
trainOrigDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_ORIG_DIR"))
trainAugmDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_AUG_DIR"))
testRootDir = os.path.commonpath([testOrigDir, testAugmDir])
trainRootDir = os.path.commonpath([trainOrigDir, trainAugmDir])
modelDir = os.path.join(projectDir, os.environ.get("MODEL_DIR"))

sys.path.append(os.path.join(projectDir,'src/data'))
sys.path.append(os.path.join(projectDir,'src/models'))
import imggen as imgen
from customlayer import DepthWiseConvBatchNorm
from customlayer import ConvBatchNorm
from customlayer import InceptionType
from customlayer import CustomInceptionFactory

from basemodel import BaseModel
from basemodel import DepthWiseConvModel1
from basemodel import ConvModel1
from basemodel import InterceptionModelV1
from basemodel import InterceptionModelV2
from basemodel import StemDecorator
from basemodel import StemDecoratorConv
from basemodel import StemDecoratorDepthWiseConv

In [11]:
trgen = imgen.Generator(trainDfPath, trainRootDir, (224,224))
trgen.SetDataFrameForGeneration([[2],[3,4]],[[2048],[1024,1024]])
train_set = trgen.GetGenerator(bs = 64)

tsgen = imgen.Generator(testDfPath,testRootDir, (224,224))
tsgen.SetDataFrameForGeneration([[2],[3,4]], [[1024],[512, 512]])
test_set = tsgen.GetGenerator(bs = 2048)

evalgen = imgen.Generator(testDfPath,testRootDir, (224,224))
evalgen.SetDataFrameForGeneration([[2],[3]], [[512],[512]])
eval_set = tsgen.GetGenerator(bs = 1024)

Found 4096 validated image filenames belonging to 2 classes.
Found 2048 validated image filenames belonging to 2 classes.
Found 2048 validated image filenames belonging to 2 classes.


In [12]:
def SaveModel(model,name,modelDir):
    path = os.path.join(modelDir,name)
    for suf in [''] + [str(i) for i in range(256)]:
        modified_path = path + suf
        if not os.path.exists(modified_path):
            os.makedirs(modified_path)
            path = modified_path
            break
    model.save(path)
    return path

In [13]:
model1 = DepthWiseConvModel1()
model2 = ConvModel1()

In [ ]:
histories = dict()
for obj in [model1, model2]:
    for unit in [512, 1024]:
        for lr in [0.0001, 0.0005, 0.001]:
            obj.units = unit
            obj.learn_rate = lr
            model = obj.Create()
            history = model.fit(train_set,
                 steps_per_epoch = 64,
                 epochs = 120, validation_data = test_set,validation_steps = 1)
            SaveModel(model, model.name, modelDir)
            histories[model.name] = history    

Model DepthWiseConvModel1_F32_U512_LR0001_LCnt6 is built
Train for 64 steps, validate for 1 steps
Epoch 1/120
63/64 [============================>.] - ETA: 5s - loss: 0.6855 - accuracy: 0.5880 

In [ ]:
intercept = StemDecoratorConv(conv_layer_cnt = 2, intercept_module = InterceptionModelV2())
model = intercept.Create()

In [ ]:
hist = model.fit(train_set,
                 steps_per_epoch = 128 / 8,
                 epochs = 1, validation_data = test_set,validation_steps = 2)

In [ ]:
m1 = ConvModel1()
model1 = m1.Create()
model1.summary()

In [ ]:
m2 = DepthWiseConvModel1()
model2 = m2.Create()
model2.summary()